# Project : "Saturday"

Hey there, this is my first pet project outside my corporate job. After moving to a new country, I realized that I had no work I could freely showcase without corporate approval. So, here we are!
I'm not a fan of doing pointless work, so this project is the first step in bringing one of my long-neglected ideas to life—a data solution for small businesses that's as high-end and user-friendly as it gets.

### The Problem We're Tackling:

- Every small business eventually faces the need to use data to improve efficiency. But implementing data into a company usually starts with hiring an expensive data wizard who designs a relational database, creates dashboards, and gets into data engineering. Often, companies realize the need for data later than they'd like. They end up regretting not making their business transparent from the start. Well, we get it, early-stage data pros are just too pricey, and many small business owners don't even know what these data folks do. So, realization often comes too late. 

- Small businesses often share similar database designs, whether it's a restaurant, a tech gadget shop, or a massage parlor. The basic data structure is quite alike. All restaurants have similar expenses and problems, just like retail stores and other small businesses. 

### 
The Project Idea:

- Create a service that's easy to implement in any business, one that uses pre-made templates to gather data and provide dashboards and numbers—essentially, "Data Analytics on a Silver Platter for Small Businesses." 

- Business owners select the data blocks they need: "Personnel, Shift Tracking, and Payroll," "Orders and Comprehensive Analysis," "Recipes for Food Preparation," "Inventory with Ingredients." For example, in a restaurant context, after gathering enough data, this could generate financial statements. Over time, we'll add more ways to gather data from different order tracking services. The goal is to simplify the lives of small business owners, taking care of what they don't understand. 


The core idea is fantastic, but it requires an enormous variety of modules. In this project, I'll dissect an example of a similar business, demonstrating my skills to future employers (I'm doing this to find a job, after all).

I'll name the project "Saturday" because in Israel, my home country, it's customary to do nothing on Shabbat. This project, on the contrary, will do everything for you. (Any resemblances to company names named after other days of the week are purely coincidental.)

# "Saturday" Case 1 

## Step 1: Designing a Relational Database

The most straightforward one for me, a restaurant. To start, we must understand and document all the data that could possibly exist in a restaurant and immediately make them all fancy and user-friendly for future use.

### First Table - Orders

In a restaurant, you can order food for dine-in or delivery. The data the restaurant has from in-house sales includes:

- order_id

- item_id - What's sold? Looking ahead, I'd also tie in here "what it's made from." It will also be a secondary key that links to the primary key in the recipe table. This would allow us to track inventory, predict consumption, form delivery sizes, and calculate the production cost of dishes.

- quantity - How much of it is sold

- cust_id - To whom it's sold (in case there's a customer database (CRM) and the ability to identify the customer, through the restaurant's mobile app, or perhaps a "Loyalty Program Card")

Additionally, if our restaurant offers delivery, we need two more variables:

- A boolean variable "Delivery" (to understand whether the order was placed in the restaurant or not)

- The delivery address if the order is for delivery. (this will also be in a secondary key format, which will link to the primary key in the addresses table - let's think about data normalization right away)

- Well, looking at all this data, we realize that any of these, on their own, wouldn't be very convenient to use as a primary key for the order. Since an order can have various items, and, as a result, the order number may repeat, I'll add a primary key - "row."

### Customers and their addresses

Similarly, I've designed a table with information about customers and addresses, linked to the main table via an ID. The table looks like this:(QuickDBD was used to create the schema)

<img src="images/schema1.png" width="" align="center" />


In [3]:
import snowflake.connector
from config import USER, PASSWORD, ACCOUNT

ctx = snowflake.connector.connect(
    user=USER,
    password=PASSWORD,
    account=ACCOUNT
)

cs = ctx.cursor()


In [34]:
cs.execute("""
CREATE DATABASE saturday
""")

print("DATABASE 'saturday' created successfully.")


DATABASE 'saturday' created successfully.


In [35]:
cs.execute("""
CREATE SCHEMA saturday.restaurant
""")

print("SCHEMA 'restaurant' created successfully.")

SCHEMA 'restaurant' created successfully.


In [37]:
# Create the 'customer' table
cs.execute("""
CREATE TABLE saturday.restaurant.customer (
    cust_id INT PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    second_name VARCHAR(50) NOT NULL
);
""")
print("TABLE 'customer' created successfully.")

# Create the 'orders' table
cs.execute("""
CREATE TABLE saturday.restaurant.orders (
    row_id INT PRIMARY KEY,
    order_id VARCHAR(50) NOT NULL,
    quantity INT NOT NULL,
    created_at DATETIME NOT NULL,
    item_id INT NOT NULL,
    cust_id INT NOT NULL,
    delivery BOOLEAN NOT NULL,
    addy_id INT NOT NULL
);
""")
print("TABLE 'orders' created successfully.")

# Create the 'address' table
cs.execute("""
CREATE TABLE saturday.restaurant.address (
    addy_id INT PRIMARY KEY,
    addy1 VARCHAR(100) NOT NULL,
    addy2 VARCHAR(100),
    addy3 VARCHAR(100),
    city VARCHAR(50) NOT NULL,
    zipcode VARCHAR(50) NOT NULL
);
""")
print("TABLE 'address' created successfully.")


TABLE 'customer' created successfully.
TABLE 'orders' created successfully.
TABLE 'address' created successfully.


### Item Information

Now, there's essential information about who and where the order is going, but no information about the item sold. This will be linked to the item_id. For optimization, I've separated information about the item and its recipe for future ease of modifications and simplified table management. The new schema looks like this:

<img src="images/schema2.png" width="" align="center" />


In [39]:
# Create the 'item' table
cs.execute("""
CREATE TABLE saturday.restaurant.item (
    item_id INT PRIMARY KEY,
    item_name VARCHAR(50) NOT NULL,
    item_cat VARCHAR(50) NOT NULL,
    item_size VARCHAR(50) NOT NULL,
    item_price DECIMAL(10,2) NOT NULL,
    recipe_id VARCHAR(20) NOT NULL
);
""")
print("TABLE 'item' created successfully.")

# Create the 'recipe' table
cs.execute("""
CREATE TABLE saturday.restaurant.recipe (
    row_id INT PRIMARY KEY,
    recipe_id VARCHAR(20) NOT NULL,
    ing_id VARCHAR(20) NOT NULL,
    quantity INT NOT NULL
);
""")
print("TABLE 'recipe' created successfully.")


TABLE 'item' created successfully.
TABLE 'recipe' created successfully.


### Stock keeping

Now we have information about the components of each product, but we still lack sufficient data to calculate the cost of each item based on ingredient prices. Additionally, we are unable to track the remaining stock on hand. To address this, we need to create two tables: "inventory" and "ingredient." The current schema looks like this:

<img src="images/schema3.png" width="" align="center" />

In [51]:
# Create the 'ingridient' table
cs.execute("""
CREATE TABLE saturday.restaurant.ingredient (
    ing_id VARCHAR(20) PRIMARY KEY,
    ing_name VARCHAR(50) NOT NULL,
    ing_weight INT NOT NULL,
    ing_meas VARCHAR(20) NOT NULL,
    ing_price DECIMAL(10,2) NOT NULL
);
""")
print("TABLE 'ingredient' created successfully.")

# Create the 'inventory' table
cs.execute("""
CREATE TABLE saturday.restaurant.inventory (
    row_id INT PRIMARY KEY,
    ing_id VARCHAR(20) NOT NULL,
    quantity INT NOT NULL
);
""")
print("TABLE 'inventory' created successfully.")


TABLE 'ingredient' created successfully.
TABLE 'inventory' created successfully.


### Salary

Now that we have information about all the production components and ingredients for each item, the missing piece for calculating their true cost is labor. To address this, we will create tables for scheduling, employees, and information about their shifts.

<img src="images/schema4.png" width="" align="center" />


In [41]:
# Create the 'schedule' table
cs.execute("""
CREATE TABLE saturday.restaurant.schedule (
    row_id INT PRIMARY KEY,
    sch_id VARCHAR(20) NOT NULL,
    date DATETIME NOT NULL,
    shift_id VARCHAR(20) NOT NULL,
    staff_id VARCHAR(20) NOT NULL
);
""")
print("TABLE 'schedule' created successfully.")

# Create the 'staff' table
cs.execute("""
CREATE TABLE saturday.restaurant.staff (
    staff_id VARCHAR(20) PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    position VARCHAR(20) NOT NULL,
    hourly_rate DECIMAL(5,2) NOT NULL
);
""")
print("TABLE 'staff' created successfully.")

# Create the 'shift' table
cs.execute("""
CREATE TABLE saturday.restaurant.shift (
    shift_id VARCHAR(20) PRIMARY KEY,
    day_of_week VARCHAR(10) NOT NULL,
    start_time TIME NOT NULL,
    end_time TIME NOT NULL
);
""")
print("TABLE 'shift' created successfully.")


TABLE 'schedule' created successfully.
TABLE 'staff' created successfully.
TABLE 'shift' created successfully.


## Step 2: Creating Queries for Dashboards

To create dashboards, we need to first process the data into a convenient format. For this, we will write several custom queries that correspond to the data we want to eventually see on the dashboard, specifically - information about orders and inventory.

### Orders

Beginning with the information about orders, we will consolidate the data into a single query for the dashboard. The query is as follows:

In [45]:
cs.execute("""
CREATE SECURE VIEW orders_view AS
SELECT
    o.order_id,
    i.item_price,
    o.quantity,
    i.item_cat,
    i.item_name,
    o.created_at,
    a.addy1,
    a.addy2,
    a.city,
    a.zipcode,
    o.delivery 
FROM
    orders o
    LEFT JOIN item i ON o.item_id = i.item_id
    LEFT JOIN address a ON o.addy_id = a.addy_id;
  
""")
print("SECURE VIEW 'orders_view' created successfully.")


SECURE VIEW 'orders_view' created successfully.


### Inventory

This part is more complex than the orders section. We need to figure out how much stock we're using and which items need to be ordered again. We also plan to work out the cost of making each pizza from the ingredient costs, to help keep track of pricing and profits.


For the dashboard, we'll gather the following data related to inventory management:

- Aggregate Quantity of Each Ingredient

- Overall Cost of All Ingredients

- Computed Cost for Each Item

- Remaining Stock as a Percentage for Each Ingredient

- Compilation of Ingredients to Reorder, Determined by Remaining Inventory

For this stage, I will divide it into two queries. In the first part, we will calculate the usage of our resources, and in the second part, we will focus on the remaining stock.

In [55]:
cs.execute("""
CREATE SECURE VIEW stock1 AS
SELECT 
    s1.item_name,
    s1.ing_id,
    s1.ing_name,
    s1.ing_weight,
    s1.ing_price,
    s1.order_quantity,
    s1.recipe_quantity,
    s1.order_quantity * s1.recipe_quantity AS ordered_weight,
    s1.ing_price/s1.ing_weight as unit_cost,
    (s1.order_quantity * s1.recipe_quantity)*(s1.ing_price/s1.ing_weight) as ingredient_cost
FROM 
    (SELECT
        o.item_id,
        i.recipe_id,
        i.item_name,
        r.ing_id,
        ing.ing_name,
        r.quantity AS recipe_quantity,
        SUM(o.quantity) AS order_quantity,
        ing.ing_weight,
        ing.ing_price
    FROM
        orders o
        LEFT JOIN item i ON o.item_id = i.item_id
        LEFT JOIN recipe r ON i.recipe_id = r.recipe_id
        LEFT JOIN ingredient ing ON ing.ing_id = r.ing_id
    GROUP BY
        o.item_id,
        i.recipe_id,
        i.item_name,
        r.ing_id,
        r.quantity,
        ing.ing_name,
        ing.ing_weight,
        ing.ing_price) s1;
""")
print("SECURE VIEW 'stock1' created successfully.")

SECURE VIEW 'stock1' created successfully.


In [57]:
cs.execute("""
CREATE VIEW stock2 AS
SELECT
    s2.ing_name,
    s2.ordered_weight,
    ing.ing_weight,
    inv.quantity,
    ing.ing_weight * inv.quantity AS total_inv_weight 
FROM
    (SELECT ing_id, ing_name, SUM(ordered_weight) AS ordered_weight FROM stock1 GROUP BY ing_name, ing_id) s2
    LEFT JOIN inventory inv ON inv.ing_id = s2.ing_id
    LEFT JOIN ingredient ing ON ing.ing_id = s2.ing_id;
""")
print("SECURE VIEW 'stock2' created successfully.")

SECURE VIEW 'stock2' created successfully.
